In [1]:
import re
from   random import *
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import os
from numpy.random import default_rng

In [2]:
# os.environ["CUDA_VISIBLE_DEVICES"] = "2"

# os.environ['http_proxy']  = 'http://192.41.170.23:3128'
# os.environ['https_proxy'] = 'http://192.41.170.23:3128'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# 1. Data

In [ ]:
from datasets import load_dataset

# dataset = load_dataset("wikipedia", "20220301.en")
# dataset

c:\Users\Bidhan\anaconda3\envs\ait\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Bidhan\anaconda3\envs\ait\Lib\site-packages\datasets\load.py:1491: FutureWarning: The repository for wikipedia contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/wikipedia
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 6458670
    })
})

In [ ]:
# dataset['train'][0] # check the first example

{'id': '12',
 'url': 'https://en.wikipedia.org/wiki/Anarchism',
 'title': 'Anarchism',
 'text': 'Anarchism is a political philosophy and movement that is sceptical of authority and rejects all involuntary, coercive forms of hierarchy. Anarchism calls for the abolition of the state, which it holds to be unnecessary, undesirable, and harmful. As a historically left-wing movement, placed on the farthest left of the political spectrum, it is usually described alongside communalism and libertarian Marxism as the libertarian wing (libertarian socialism) of the socialist movement, and has a strong historical association with anti-capitalism and socialism.\n\nHumans lived in societies without formal hierarchies long before the establishment of formal states, realms, or empires. With the rise of organised hierarchical bodies, scepticism toward authority also rose. Although traces of anarchist thought are found throughout history, modern anarchism emerged from the Enlightenment. During the latte

In [ ]:
# # random numnber generator
# rand = default_rng(SEED)

# # random index from the dataset
# # reducing the size of the dataset to 100k
# random_index = rand.choice(len(dataset['train']), 100000, replace=False)

# dataset['train'] = dataset['train'].filter(lambda data, index: index in random_index, with_indices=True)

In [ ]:
# dataset # after filtering

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 100000
    })
})

In [ ]:
# removing unwanted columns
# dataset = dataset.remove_columns(['id', 'url', 'title'])

In [ ]:
# dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 100000
    })
})

In [ ]:
# dataset['train'][:5]

{'text': ['Allah (; , ) is the common Arabic word for God. In the English language, the word generally refers to God in Islam. The word is thought to be derived by contraction from al-ilāh, which means "the god", and is linguistically related to the  Aramaic words Elah and  (ʼAlâhâ) and the Hebrew word El (Elohim) for God.\n\nThe word Allah has been used by Arabic people of different religions since pre-Islamic times. The pre-Islamic Arabs worshipped a supreme deity whom they called Allah, alongside other lesser deities. Muhammad used the word Allah to indicate the Islamic conception of God. Allah has been used as a term for God by Muslims (both Arab and non-Arab) and even Arab Christians after the term "al-ilāh" and "Allah" were used interchangeably in Classical Arabic by the majority of Arabs who had become Muslims. It is also often, albeit not exclusively, used in this way by Bábists, Baháʼís, Mandaeans, Indonesian and Maltese Christians, and Sephardi Jews. Similar usage by Christia

In [ ]:
# sentences = dataset['train']['text']
# sentences = [x.replace("\n", " ") for x in sentences]
# sentences = [x for x in sentences if len(x.split()) <= 500]
# sentences[:5]

['Anaxarchus (; ; ) was a Greek philosopher of the school of Democritus. Together with Pyrrho, he accompanied Alexander the Great into Asia. The reports of his philosophical views suggest that he was a forerunner of Pyrrhonism. Aelian writes that he was called Eudaemonicus or "Happy Man" ().  Life   Anaxarchus was born at Abdera in Thrace. He was the companion and friend of Alexander the Great in his Asiatic campaigns. According to Diogenes Laërtius, in response to Alexander\'s claim to have been the son of Zeus-Ammon, Anaxarchus pointed to his bleeding wound and remarked, "See the blood of a mortal, not ichor, such as flows from the veins of the immortal gods." Aelian, writes that Anaxarchus laughed at Alexander for making himself a god and said, "The hopes of our god are in a porringer of broth", when the physician prescribed a broth to Alexander.  Plutarch tells a story that at Bactra, in 327\xa0BC in a debate with Callisthenes, he advised all to worship Alexander as a god even duri

### Making vocabs

Before making the vocabs, let's remove all question marks and perios, etc, then turn everything to lowercase, and then simply split the text. 

In [12]:
text = [x.lower() for x in sentences] #lower case
text = [re.sub("[.,!?\\-]", '', x) for x in text] #clean all symbols
text[:5]

['anaxarchus (; ; ) was a greek philosopher of the school of democritus together with pyrrho he accompanied alexander the great into asia the reports of his philosophical views suggest that he was a forerunner of pyrrhonism aelian writes that he was called eudaemonicus or "happy man" ()  life   anaxarchus was born at abdera in thrace he was the companion and friend of alexander the great in his asiatic campaigns according to diogenes laërtius in response to alexander\'s claim to have been the son of zeusammon anaxarchus pointed to his bleeding wound and remarked "see the blood of a mortal not ichor such as flows from the veins of the immortal gods" aelian writes that anaxarchus laughed at alexander for making himself a god and said "the hopes of our god are in a porringer of broth" when the physician prescribed a broth to alexander  plutarch tells a story that at bactra in 327\xa0bc in a debate with callisthenes he advised all to worship alexander as a god even during his lifetime as t

In [13]:
from tqdm.auto import tqdm

# Combine everything into one to make vocab
word_list = list(set(" ".join(text).split()))
word2id = {'[PAD]': 0, '[CLS]': 1, '[SEP]': 2, '[MASK]': 3}  # special tokens

# Create the word2id in a single pass
for i, w in tqdm(enumerate(word_list), desc="Creating word2id"):
    word2id[w] = i + 4  # because 0-3 are already occupied

# Precompute the id2word mapping (this can be done once after word2id is fully populated)
id2word = {v: k for k, v in word2id.items()}
vocab_size = len(word2id)
vocab_size

Creating word2id: 598003it [00:00, 2529390.02it/s]


598007

In [14]:
# List of all tokens for the whole text
token_list = []

# Process sentences more efficiently
for sentence in tqdm(text, desc="Processing sentences"):
    token_list.append([word2id[word] for word in sentence.split()])

Processing sentences: 100%|██████████| 74796/74796 [00:02<00:00, 27755.36it/s]


In [15]:
#take a look at sentences
sentences[:2]

['Anaxarchus (; ; ) was a Greek philosopher of the school of Democritus. Together with Pyrrho, he accompanied Alexander the Great into Asia. The reports of his philosophical views suggest that he was a forerunner of Pyrrhonism. Aelian writes that he was called Eudaemonicus or "Happy Man" ().  Life   Anaxarchus was born at Abdera in Thrace. He was the companion and friend of Alexander the Great in his Asiatic campaigns. According to Diogenes Laërtius, in response to Alexander\'s claim to have been the son of Zeus-Ammon, Anaxarchus pointed to his bleeding wound and remarked, "See the blood of a mortal, not ichor, such as flows from the veins of the immortal gods." Aelian, writes that Anaxarchus laughed at Alexander for making himself a god and said, "The hopes of our god are in a porringer of broth", when the physician prescribed a broth to Alexander.  Plutarch tells a story that at Bactra, in 327\xa0BC in a debate with Callisthenes, he advised all to worship Alexander as a god even duri

In [16]:
#take a look at token_list
token_list[:2]

[[360865,
  529878,
  59477,
  548158,
  64798,
  227268,
  542256,
  56011,
  459158,
  272249,
  72413,
  459158,
  417171,
  89317,
  415301,
  337138,
  366214,
  569962,
  404335,
  272249,
  163521,
  405675,
  390923,
  272249,
  23228,
  459158,
  384991,
  20569,
  225257,
  351533,
  68596,
  366214,
  64798,
  227268,
  594717,
  459158,
  339137,
  250315,
  291368,
  68596,
  366214,
  64798,
  111510,
  345933,
  403278,
  371049,
  482337,
  232809,
  95904,
  360865,
  64798,
  279797,
  90398,
  308599,
  156750,
  503213,
  366214,
  64798,
  272249,
  588881,
  529837,
  114086,
  459158,
  404335,
  272249,
  163521,
  156750,
  384991,
  143797,
  574458,
  92623,
  175989,
  503481,
  371677,
  156750,
  8409,
  175989,
  476747,
  212994,
  175989,
  243187,
  345700,
  272249,
  566704,
  459158,
  559694,
  360865,
  405540,
  175989,
  384991,
  193108,
  345514,
  529837,
  205375,
  414601,
  272249,
  151784,
  459158,
  227268,
  5120,
  117353,
  183811,


In [17]:
#testing one sentence
for tokens in token_list[0]:
    print(id2word[tokens])

anaxarchus
(;
;
)
was
a
greek
philosopher
of
the
school
of
democritus
together
with
pyrrho
he
accompanied
alexander
the
great
into
asia
the
reports
of
his
philosophical
views
suggest
that
he
was
a
forerunner
of
pyrrhonism
aelian
writes
that
he
was
called
eudaemonicus
or
"happy
man"
()
life
anaxarchus
was
born
at
abdera
in
thrace
he
was
the
companion
and
friend
of
alexander
the
great
in
his
asiatic
campaigns
according
to
diogenes
laërtius
in
response
to
alexander's
claim
to
have
been
the
son
of
zeusammon
anaxarchus
pointed
to
his
bleeding
wound
and
remarked
"see
the
blood
of
a
mortal
not
ichor
such
as
flows
from
the
veins
of
the
immortal
gods"
aelian
writes
that
anaxarchus
laughed
at
alexander
for
making
himself
a
god
and
said
"the
hopes
of
our
god
are
in
a
porringer
of
broth"
when
the
physician
prescribed
a
broth
to
alexander
plutarch
tells
a
story
that
at
bactra
in
327
bc
in
a
debate
with
callisthenes
he
advised
all
to
worship
alexander
as
a
god
even
during
his
lifetime
as
they
would


# 2. Data loader

We gonna make dataloader.  Inside here, we need to make two types of embeddings: **token embedding** and **segment embedding**

1. **Token embedding** - Given “The cat is walking. The dog is barking”, we add [CLS] and [SEP] >> “[CLS] the cat is walking [SEP] the dog is barking”. 

2. **Segment embedding**
A segment embedding separates two sentences, i.e., [0 0 0 0 1 1 1 1 ]

3. **Masking**
As mentioned in the original paper, BERT randomly assigns masks to 15% of the sequence. In this 15%, 80% is replaced with masks, while 10% is replaced with random tokens, and the rest 10% is left as is.  Here we specified `max_pred` 

4. **Padding**
Once we mask, we will add padding. For simplicity, here we padded until some specified `max_len`. 

Note:  `positive` and `negative` are just simply counts to keep track of the batch size.  `positive` refers to two sentences that are really next to one another.

In [18]:
batch_size = 6
max_mask   = 5  # max masked tokens when 15% exceed, it will only be max_pred
max_len    = 1000 # maximum of length to be padded; 

In [19]:
def make_batch():
    batch = []
    half_batch_size = batch_size // 2
    positive = negative = 0
    while positive != half_batch_size or negative != half_batch_size:

        #randomly choose two sentence
        tokens_a_index, tokens_b_index = np.random.randint(len(sentences), size=2)
        tokens_a, tokens_b            = token_list[tokens_a_index], token_list[tokens_b_index]

        #1. token embedding - add CLS and SEP
        input_ids = [word2id['[CLS]']] + tokens_a + [word2id['[SEP]']] + tokens_b + [word2id['[SEP]']]

        #2. segment embedding - which sentence is 0 and 1
        segment_ids = [0] * (1 + len(tokens_a) + 1) + [1] * (len(tokens_b) + 1)

        n_pred = min(max_mask, max(1, int(round(len(input_ids) * 0.15))))
        #get all the pos excluding CLS and SEP
        candidates_masked_pos = [i for i, token in enumerate(input_ids) if token != word2id['[CLS]']
                                 and token != word2id['[SEP]']]
        np.random.shuffle(candidates_masked_pos)
        masked_tokens, masked_pos = [], []
        #simply loop and mask accordingly
        for pos in candidates_masked_pos[:n_pred]:
            masked_pos.append(pos)
            masked_tokens.append(input_ids[pos])
            rand_val = np.random.random()
            if rand_val < 0.1:  #10% replace with random token
                index = np.random.randint(4, vocab_size - 1)  # random token should not involve [PAD], [CLS], [SEP], [MASK]
                input_ids[pos] = word2id[id2word[index]]
            elif rand_val < 0.8:  #80 replace with [MASK]
                input_ids[pos] = word2id['[MASK]']
            else:
                pass

        #4. pad the sentence to the max length
        n_pad = max_len - len(input_ids)
        input_ids.extend([0] * n_pad)
        segment_ids.extend([0] * n_pad)

        #5. pad the mask tokens to the max length
        if max_mask > n_pred:
            n_pad = max_mask - n_pred
            masked_tokens.extend([0] * n_pad)
            masked_pos.extend([0] * n_pad)

        #6. check whether is positive or negative
        if tokens_a_index + 1 == tokens_b_index and positive < half_batch_size:
            batch.append([input_ids, segment_ids, masked_tokens, masked_pos, True])
            positive += 1
        elif tokens_a_index + 1 != tokens_b_index and negative < half_batch_size:
            batch.append([input_ids, segment_ids, masked_tokens, masked_pos, False])
            negative += 1

    return batch

In [20]:
batch = make_batch()

In [21]:
len(batch)

6

In [22]:
input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(*batch))

In [23]:
input_ids.shape, segment_ids.shape, masked_tokens.shape, masked_pos.shape, isNext.shape

(torch.Size([6, 1000]),
 torch.Size([6, 1000]),
 torch.Size([6, 5]),
 torch.Size([6, 5]),
 torch.Size([6]))

# 3. Model

Recall that BERT only uses the encoder.

BERT has the following components:

- Embedding layers
- Attention Mask
- Encoder layer
- Multi-head attention
- Scaled dot product attention
- Position-wise feed-forward network
- BERT (assembling all the components)

## 3.1 Embedding

Here we simply generate the positional embedding, and sum the token embedding, positional embedding, and segment embedding together.

In [24]:
class Embedding(nn.Module):
    def __init__(self, vocab_size, max_len, n_segments, d_model, device):
        super(Embedding, self).__init__()
        self.tok_embed = nn.Embedding(vocab_size, d_model)  # token embedding
        self.pos_embed = nn.Embedding(max_len, d_model)      # position embedding
        self.seg_embed = nn.Embedding(n_segments, d_model)  # segment(token type) embedding
        self.norm = nn.LayerNorm(d_model)
        self.device = device

    def forward(self, x, seg):
        #x, seg: (bs, len)
        seq_len = x.size(1)
        pos = torch.arange(seq_len, dtype=torch.long).to(self.device)
        pos = pos.unsqueeze(0).expand_as(x)  # (len,) -> (bs, len)
        embedding = self.tok_embed(x) + self.pos_embed(pos) + self.seg_embed(seg)
        return self.norm(embedding)

## 3.2 Attention mask

In [25]:
def get_attn_pad_mask(seq_q, seq_k, device):
    batch_size, len_q = seq_q.size()
    batch_size, len_k = seq_k.size()
    # eq(zero) is PAD token
    pad_attn_mask = seq_k.data.eq(0).unsqueeze(1).to(device)  # batch_size x 1 x len_k(=len_q), one is masking
    return pad_attn_mask.expand(batch_size, len_q, len_k)  # batch_size x len_q x len_k

### Testing the attention mask

In [26]:
print(get_attn_pad_mask(input_ids, input_ids, device).shape)

torch.Size([6, 1000, 1000])


## 3.3 Encoder

The encoder has two main components: 

- Multi-head Attention
- Position-wise feed-forward network

First let's make the wrapper called `EncoderLayer`

In [27]:
class EncoderLayer(nn.Module):
    def __init__(self, n_heads, d_model, d_ff, d_k, device):
        super(EncoderLayer, self).__init__()
        self.enc_self_attn = MultiHeadAttention(n_heads, d_model, d_k, device)
        self.pos_ffn       = PoswiseFeedForwardNet(d_model, d_ff)

    def forward(self, enc_inputs, enc_self_attn_mask):
        enc_outputs, attn = self.enc_self_attn(enc_inputs, enc_inputs, enc_inputs, enc_self_attn_mask) # enc_inputs to same Q,K,V
        enc_outputs = self.pos_ffn(enc_outputs) # enc_outputs: [batch_size x len_q x d_model]
        return enc_outputs, attn

Let's define the scaled dot attention, to be used inside the multihead attention

In [28]:
class ScaledDotProductAttention(nn.Module):
    def __init__(self, d_k, device):
        super(ScaledDotProductAttention, self).__init__()
        self.scale = torch.sqrt(torch.FloatTensor([d_k])).to(device)

    def forward(self, Q, K, V, attn_mask):
        scores = torch.matmul(Q, K.transpose(-1, -2)) / self.scale # scores : [batch_size x n_heads x len_q(=len_k) x len_k(=len_q)]
        scores.masked_fill_(attn_mask, -1e9) # Fills elements of self tensor with value where mask is one.
        attn = nn.Softmax(dim=-1)(scores)
        context = torch.matmul(attn, V)
        return context, attn 

Let's define the parameters first

In [29]:
n_layers = 6    # number of Encoder of Encoder Layer
n_heads  = 8    # number of heads in Multi-Head Attention
d_model  = 768  # Embedding Size
d_ff = 768 * 4  # 4*d_model, FeedForward dimension
d_k = d_v = 64  # dimension of K(=Q), V
n_segments = 2

Here is the Multiheadattention.

In [30]:
class MultiHeadAttention(nn.Module):
    def __init__(self, n_heads, d_model, d_k, device):
        super(MultiHeadAttention, self).__init__()
        self.n_heads = n_heads
        self.d_model = d_model
        self.d_k = d_k
        self.d_v = d_k
        self.W_Q = nn.Linear(d_model, d_k * n_heads)
        self.W_K = nn.Linear(d_model, d_k * n_heads)
        self.W_V = nn.Linear(d_model, self.d_v * n_heads)
        self.device = device
    def forward(self, Q, K, V, attn_mask):
        # q: [batch_size x len_q x d_model], k: [batch_size x len_k x d_model], v: [batch_size x len_k x d_model]
        residual, batch_size = Q, Q.size(0)
        # (B, S, D) -proj-> (B, S, D) -split-> (B, S, H, W) -trans-> (B, H, S, W)
        q_s = self.W_Q(Q).view(batch_size, -1, self.n_heads, self.d_k).transpose(1,2)  # q_s: [batch_size x n_heads x len_q x d_k]
        k_s = self.W_K(K).view(batch_size, -1, self.n_heads, self.d_k).transpose(1,2)  # k_s: [batch_size x n_heads x len_k x d_k]
        v_s = self.W_V(V).view(batch_size, -1, self.n_heads, self.d_v).transpose(1,2)  # v_s: [batch_size x n_heads x len_k x d_v]

        attn_mask = attn_mask.unsqueeze(1).repeat(1, self.n_heads, 1, 1) # attn_mask : [batch_size x n_heads x len_q x len_k]

        # context: [batch_size x n_heads x len_q x d_v], attn: [batch_size x n_heads x len_q(=len_k) x len_k(=len_q)]
        context, attn = ScaledDotProductAttention(self.d_k, self.device)(q_s, k_s, v_s, attn_mask)
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, self.n_heads * self.d_v) # context: [batch_size x len_q x n_heads * d_v]
        output = nn.Linear(self.n_heads * self.d_v, self.d_model, device=self.device)(context)
        return nn.LayerNorm(self.d_model, device=self.device)(output + residual), attn # output: [batch_size x len_q x d_model]

Here is the PoswiseFeedForwardNet.

In [31]:
class PoswiseFeedForwardNet(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PoswiseFeedForwardNet, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)

    def forward(self, x):
        # (batch_size, len_seq, d_model) -> (batch_size, len_seq, d_ff) -> (batch_size, len_seq, d_model)
        return self.fc2(F.gelu(self.fc1(x)))

## 3.4 Putting them together

In [32]:
class BERT(nn.Module):
    def __init__(self, n_layers, n_heads, d_model, d_ff, d_k, n_segments, vocab_size, max_len, device):
        super(BERT, self).__init__()
        self.params = {'n_layers': n_layers, 'n_heads': n_heads, 'd_model': d_model,
                       'd_ff': d_ff, 'd_k': d_k, 'n_segments': n_segments,
                       'vocab_size': vocab_size, 'max_len': max_len}
        self.embedding = Embedding(vocab_size, max_len, n_segments, d_model, device)
        self.layers = nn.ModuleList([EncoderLayer(n_heads, d_model, d_ff, d_k, device) for _ in range(n_layers)])
        self.fc = nn.Linear(d_model, d_model)
        self.activ = nn.Tanh()
        self.linear = nn.Linear(d_model, d_model)
        self.norm = nn.LayerNorm(d_model)
        self.classifier = nn.Linear(d_model, 2)
        # decoder is shared with embedding layer
        embed_weight = self.embedding.tok_embed.weight
        n_vocab, n_dim = embed_weight.size()
        self.decoder = nn.Linear(n_dim, n_vocab, bias=False)
        self.decoder.weight = embed_weight
        self.decoder_bias = nn.Parameter(torch.zeros(n_vocab))
        self.device = device

    def forward(self, input_ids, segment_ids, masked_pos):
        output = self.embedding(input_ids, segment_ids)
        enc_self_attn_mask = get_attn_pad_mask(input_ids, input_ids, self.device)
        for layer in self.layers:
            output, enc_self_attn = layer(output, enc_self_attn_mask)
        # output : [batch_size, len, d_model], attn : [batch_size, n_heads, d_mode, d_model]
        
        # 1. predict next sentence
        # it will be decided by first token(CLS)
        h_pooled   = self.activ(self.fc(output[:, 0])) # [batch_size, d_model]
        logits_nsp = self.classifier(h_pooled) # [batch_size, 2]

        # 2. predict the masked token
        masked_pos = masked_pos[:, :, None].expand(-1, -1, output.size(-1)) # [batch_size, max_pred, d_model]
        h_masked = torch.gather(output, 1, masked_pos) # masking position [batch_size, max_pred, d_model]
        h_masked  = self.norm(F.gelu(self.linear(h_masked)))
        logits_lm = self.decoder(h_masked) + self.decoder_bias # [batch_size, max_pred, n_vocab]

        return logits_lm, logits_nsp
    
    def get_last_hidden_state(self, input_ids, segment_ids):
        output = self.embedding(input_ids, segment_ids)
        enc_self_attn_mask = get_attn_pad_mask(input_ids, input_ids, self.device)
        for layer in self.layers:
            output, enc_self_attn = layer(output, enc_self_attn_mask)

        return output

## 4. Training

In [33]:
from tqdm.auto import tqdm

n_layers = 12    # number of Encoder of Encoder Layer
n_heads  = 12    # number of heads in Multi-Head Attention
d_model  = 768  # Embedding Size
d_ff = d_model * 4  # 4*d_model, FeedForward dimension
d_k = d_v = 64  # dimension of K(=Q), V
n_segments = 2

num_epoch = 1000
model = BERT(
    n_layers, 
    n_heads, 
    d_model, 
    d_ff, 
    d_k, 
    n_segments, 
    vocab_size, 
    max_len, 
    device
).to(device)  # Move model to GPU

In [34]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [35]:
batch = make_batch()
input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(*batch))

# Move inputs to GPU
input_ids = input_ids.to(device)
segment_ids = segment_ids.to(device)
masked_tokens = masked_tokens.to(device)
masked_pos = masked_pos.to(device)
isNext = isNext.to(device)

# Wrap the epoch loop with tqdm
for epoch in tqdm(range(num_epoch), desc="Training Epochs"):
    optimizer.zero_grad()
    logits_lm, logits_nsp = model(input_ids, segment_ids, masked_pos)    
    #logits_lm: (bs, max_mask, vocab_size) ==> (6, 5, 34)
    #logits_nsp: (bs, yes/no) ==> (6, 2)

    #1. mlm loss
    #logits_lm.transpose: (bs, vocab_size, max_mask) vs. masked_tokens: (bs, max_mask)
    loss_lm = criterion(logits_lm.transpose(1, 2), masked_tokens) # for masked LM
    loss_lm = (loss_lm.float()).mean()
    #2. nsp loss
    #logits_nsp: (bs, 2) vs. isNext: (bs, )
    loss_nsp = criterion(logits_nsp, isNext) # for sentence classification
    
    #3. combine loss
    loss = loss_lm + loss_nsp
    if epoch % 100 == 0:
        print('Epoch:', '%02d' % (epoch), 'loss =', '{:.6f}'.format(loss))
    loss.backward()
    optimizer.step()

Training Epochs:   0%|          | 0/1000 [00:00<?, ?it/s]

Epoch: 00 loss = 147.895966


Training Epochs:   0%|          | 1/1000 [00:33<9:17:09, 33.46s/it]


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.71 GiB. GPU 

In [ ]:
# Save the model after training
torch.save(model.state_dict(), 'model/bert_model.pth')
print("Model saved to bert_model.pth")